In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf

df = pd.read_csv('s2-in.txt', sep='\t')
df.head()
text_combined = df['Text'].str.cat(sep=' ')

In [2]:
# build dictionary v. 1.0, maybe unnecessary? 完全不必要，用Keras的Tokenizer就行了
vocab = {}
i = 1

for word in text_combined.split():
    if word in vocab:
        continue
    else:
        vocab[word]=i
        i+=1

In [3]:
# separator v. 1.0
def tokenize(text):
    return text.split()
tokens = tokenize(text_combined)
len(tokens)
tokens[:10]

['WMLK>',
 'DWJD',
 'S>B',
 'W<L',
 'BC"NJ>',
 'WMKSJN',
 'HWW',
 'LH',
 'BLBWC">',
 'WL>']

In [4]:
# organize into sequences of tokens
train_len = 25+1 # 25 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [5]:
# Tokenize the sequences
import keras
from tensorflow.keras.preprocessing.text import Tokenizer

# integer encode sequences of words
tokenizer = Tokenizer(lower=False) #ETCBC data is in uppercase, we need to keep it that way
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)
vocabulary_size = len(tokenizer.word_index) #总共有多少词形
vocabulary_size

18034

In [6]:
import numpy as np
sequences = np.array(sequences) #converts the Python list into a NumPy array.

In [7]:
#formalize the data and labels. The data will be all the words in the sequence except the last one, and the label will be the last word in the sequence.

import tensorflow as tf
from keras.utils import to_categorical

X = sequences[:,:-1] #all rows, all columns except the last one
y = sequences[:,-1] #all rows, last column only
y_one_hot = to_categorical(y, num_classes=vocabulary_size+1) #one-hot encoding
seq_len = X.shape[1]
y_one_hot.shape
seq_len

25

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
def creat_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

model = creat_model(vocabulary_size+1, seq_len)

2024-05-05 15:18:05.692410: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-05 15:18:05.833363: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-05 15:18:05.833486: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
from pickle import dump, load
model.fit(X, y_one_hot, batch_size=128, epochs=500, verbose=1)

Epoch 1/500


2024-05-05 15:18:12.719073: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


610/610 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.0201 - loss: 8.8215
Epoch 2/500
610/610 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.0226 - loss: 8.1637
Epoch 3/500
375/610 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.0238 - loss: 8.0367

KeyboardInterrupt: 

In [ ]:
model.save('s2-out.h5')
dump(tokenizer, open('s2-out', 'wb'))

NameError: name 'model' is not defined

In [ ]:
from keras.models import load_model
from pickle import load
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict class probabilities for each word
        pred_probabilities = model.predict(pad_encoded, verbose=0)[0]

        # Get the index of the word with the highest probability
        pred_word_ind = np.argmax(pred_probabilities)
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [ ]:

seed_text = "DNTL LK >R<> DMRDJ> XLB> WDBC> CM< >JSRJL MRJ> >LHN MRJ> XD HW RXM LMRJ> >LHK MN KLH LBK WMN KLH^ NPCK WMN KLH QNJNK"

len(seed_text.split())
model = load_model('s2-out.h5')
tokenizer = load(open('s2-out', 'rb'))

generate_text(model,tokenizer,seq_len=25,seed_text=seed_text,num_gen_words=25)

2024-03-31 08:28:00.416074: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-31 08:28:00.416683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-31 08:28:00.417400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

'LJV MN DKTJB WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ>'